In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import arviz as az
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy.stats as st
from tqdm import trange

from functions.load_data import load_data
from functions.gibbs_reg import gibbs_regression
from functions.mcmc_stats import mcmc_stats
from code.functions.mcmc_plots_posterior import mcmc_plots
from functions.plot_posterior_vs_prior import prior_vs_posterior_plot
from functions.sddr_beta import sddr_beta

data = load_data()

/Users/satoshan/.pyenv/versions/miniforge3-4.10.3-10/envs/numpyro-env/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


$$\pi_t = \alpha \pi_{t-1} + \beta E_{t}\pi_{t+1} + \kappa Y_{t} + \gamma \Delta\text{oil}_{t} + \varepsilon_t$$

$E_{t}\pi_{t+1}$:

head line inflation expectation (model’s output)

$Y_t$: Output gap


$\Delta\text{oil}$: 

Year on year change in log oil prices (WTI oil price index), Aggregated method: Avarage

In [2]:
# ==== priors ====
tau  = 1.0
nu0  = 12.0
lam0 = 12.0

# ==== data ====
targets   = ["pi_cpi", "pi_cpi_core", "pi_pce", "pi_pce_core"]  
epi_specs = ["Epi", "Epi_spf_gdp", "Epi_spf_cpi"]  
def _lag_var_for(target):
    return f"{target}_prev"

def _prepare_X_y(df, target, epi_col, output_var):
    lag_col = _lag_var_for(target)
    cols = [lag_col, epi_col, output_var, "N_BN", "oil"]
    tmp = df[[target] + cols].dropna().copy()
    y = tmp[target].to_numpy()
    X = tmp[cols].copy()
    return X, y, tmp.index, cols

def _ols_fit(X, y):
    model = sm.OLS(y, X)  
    res = model.fit()
    out = (pd.DataFrame({
        "coef": res.params,
        "std_err": res.bse,
        "t": res.tvalues,
        "p": res.pvalues
    })
           .reset_index()
           .rename(columns={"index": "param"}))
    return out, res

def _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234):
    k = X.shape[1]
    A0 = np.eye(k) / (tau**2)
    b0 = np.zeros(k)
    hyper_param = [b0, A0, nu0, lam0]
    runs = gibbs_regression(y, X.to_numpy(), hyper_param, burnin, draws, rng=np.random.default_rng(seed))
    beta = runs[burnin:, :k]
    mean = beta.mean(0)
    lo, hi = np.quantile(beta, [0.025, 0.975], axis=0)
    out = pd.DataFrame({
        "param": X.columns,
        "post_mean": mean,
        "ci_lower": lo,
        "ci_upper": hi
    })
    return out, runs


In [3]:
ols_rows   = []
gibbs_rows = []

for target in targets:
    for epi in epi_specs:
        X, y, used_idx, cols = _prepare_X_y(data, target, "output_gap_BN", epi)
        # OLS
        ols_tbl, _ = _ols_fit(X, y)
        ols_tbl.insert(0, "target", target)
        ols_tbl.insert(1, "Epi_spec", epi)
        ols_rows.append(ols_tbl)
        # Gibbs
        gibbs_tbl, runs = _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234)
        gibbs_tbl.insert(0, "target", target)
        gibbs_tbl.insert(1, "Epi_spec", epi)
        gibbs_rows.append(gibbs_tbl)

ols_results   = pd.concat(ols_rows, ignore_index=True)
gibbs_results = pd.concat(gibbs_rows, ignore_index=True)
ols_results   = ols_results[["target","Epi_spec","param","coef","std_err","t","p"]]
gibbs_results = gibbs_results[["target","Epi_spec","param","post_mean","ci_lower","ci_upper"]]

print("=== OLS: output_gap_BN kappa ===")
ols_gap_coef = (
    ols_results.query("param == 'output_gap_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'output_gap_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)
print("=== OLS: output_gap_BN theta ===")
ols_gap_coef = (
    ols_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)


print("\n=== Gibbs results (posterior mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'output_gap_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

print("\n=== Gibbs results (posterior theta mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

100%|██████████| 22000/22000 [00:01<00:00, 18316.42it/s]


=== OLS: output_gap_BN kappa ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef     -0.023       0.017       0.009       0.016       0.041       0.028   
p         0.458       0.577       0.764       0.464       0.041       0.138   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef     -0.023      -0.002       0.000      -0.007       0.001       0.004  
p         0.274       0.920       0.999       0.626       0.947       0.761

=== OLS: output_gap_BN theta ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.910      -0.127       0.203       0.313      -0.230       0.028   
p         0.181       0.860       0.763       0.474       0.585       0.944   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef      1.295       0.487       0.808       0.606       0.339       0.427  
p         0.009       0.315       0.093       0.058       0.276       0.165


=== Gibbs results (posterior mean) ===


target        pi_cpi                         pi_cpi_core              \
Epi_spec         Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp   
param                                                                  
output_gap_BN -0.016       0.019       0.012       0.017        0.04   

target                    pi_pce                         pi_pce_core  \
Epi_spec      Epi_spf_cpi    Epi Epi_spf_gdp Epi_spf_cpi         Epi   
param                                                                  
output_gap_BN       0.028 -0.017       0.001       0.004      -0.004   

target                                 
Epi_spec      Epi_spf_gdp Epi_spf_cpi  
param                                  
output_gap_BN       0.002       0.005


=== Gibbs results (posterior theta mean) ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
param                                                                         
N_BN      0.517      -0.115       0.085       0.228      -0.168       0.023   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
param                                                                        
N_BN      0.882       0.318       0.545        0.46       0.261       0.328

In [4]:
ols_rows   = []
gibbs_rows = []

for target in targets:
    for epi in epi_specs:
        X, y, used_idx, cols = _prepare_X_y(data, target, "unemp_gap", epi)
        # OLS
        ols_tbl, _ = _ols_fit(X, y)
        ols_tbl.insert(0, "target", target)
        ols_tbl.insert(1, "Epi_spec", epi)
        ols_rows.append(ols_tbl)
        # Gibbs
        gibbs_tbl, runs = _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234)
        gibbs_tbl.insert(0, "target", target)
        gibbs_tbl.insert(1, "Epi_spec", epi)
        gibbs_rows.append(gibbs_tbl)

ols_results   = pd.concat(ols_rows, ignore_index=True)
gibbs_results = pd.concat(gibbs_rows, ignore_index=True)
ols_results   = ols_results[["target","Epi_spec","param","coef","std_err","t","p"]]
gibbs_results = gibbs_results[["target","Epi_spec","param","post_mean","ci_lower","ci_upper"]]

print("=== OLS: unemp_gap kappa ===")
ols_gap_coef = (
    ols_results.query("param == 'unemp_gap'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'unemp_gap'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("=== OLS: unemp_gap theta ===")
ols_gap_coef = (
    ols_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("\n=== Gibbs results (posterior kappa mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'unemp_gap'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)
print("\n=== Gibbs results (posterior theta mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

100%|██████████| 22000/22000 [00:00<00:00, 23101.12it/s]

=== OLS: unemp_gap kappa ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.002       0.050       0.033       0.023       0.051       0.040   
p         0.925       0.054       0.160       0.117       0.001       0.005   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef     -0.036      -0.002      -0.016      -0.011       0.002      -0.003  
p         0.037       0.906       0.346       0.329       0.866       0.799

=== OLS: unemp_gap theta ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.784      -0.116       0.225       0.423      -0.035       0.199   
p         0.235       0.868       0.730       0.288       0.929       0.596   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef      1.147       0.476       0.812       0.576       0.343       0.461  
p         0.014       0.311       0.079       0.052       0.243       0.114


=== Gibbs results (posterior kappa mean) ===


target    pi_cpi                         pi_cpi_core                          \
Epi_spec     Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
param                                                                          
unemp_gap  0.003        0.05       0.034       0.023       0.051        0.04   

target    pi_pce                         pi_pce_core                          
Epi_spec     Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
param                                                                         
unemp_gap -0.035      -0.001      -0.015      -0.011       0.002      -0.003


=== Gibbs results (posterior theta mean) ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
param                                                                         
N_BN      0.459      -0.102        0.11       0.317      -0.028        0.15   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
param                                                                        
N_BN      0.807       0.321       0.565       0.451        0.27       0.362

In [5]:
ols_rows   = []
gibbs_rows = []

for target in targets:
    for epi in epi_specs:
        X, y, used_idx, cols = _prepare_X_y(data, target, "markup_BN_inv", epi)
        # OLS
        ols_tbl, _ = _ols_fit(X, y)
        ols_tbl.insert(0, "target", target)
        ols_tbl.insert(1, "Epi_spec", epi)
        ols_rows.append(ols_tbl)
        # Gibbs
        gibbs_tbl, runs = _gibbs_fit(X, y, draws=20000, burnin=2000, seed=1234)
        gibbs_tbl.insert(0, "target", target)
        gibbs_tbl.insert(1, "Epi_spec", epi)
        gibbs_rows.append(gibbs_tbl)

ols_results   = pd.concat(ols_rows, ignore_index=True)
gibbs_results = pd.concat(gibbs_rows, ignore_index=True)
ols_results   = ols_results[["target","Epi_spec","param","coef","std_err","t","p"]]
gibbs_results = gibbs_results[["target","Epi_spec","param","post_mean","ci_lower","ci_upper"]]

print("=== OLS: markup_BN_inv ===")
ols_gap_coef = (
    ols_results.query("param == 'markup_BN_inv'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'markup_BN_inv'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("=== OLS: markup theta ===")
ols_gap_coef = (
    ols_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="coef")
    .round(3)
)
ols_gap_p = (
    ols_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="p")
    .round(3)
)
ols_gap_coef.index = ["coef"]
ols_gap_p.index    = ["p"]
ols_gap_two_rows = pd.concat([ols_gap_coef, ols_gap_p])
display(ols_gap_two_rows)

print("\n=== Gibbs results (posterior kappa mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'markup_BN_inv'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)
print("\n=== Gibbs results (posterior theta mean) ===")
gibbs_gap = (
    gibbs_results.query("param == 'N_BN'")
    .pivot(index="param", columns=["target","Epi_spec"], values="post_mean")
    .round(3)
)
display(gibbs_gap)

100%|██████████| 22000/22000 [00:00<00:00, 23120.88it/s]

=== OLS: markup_BN_inv ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.030       0.088       0.078       0.038       0.073       0.071   
p         0.638       0.201       0.225       0.340       0.085       0.070   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef     -0.049      -0.008      -0.025      -0.009       0.006      -0.001  
p         0.283       0.866       0.577       0.758       0.844       0.979

=== OLS: markup theta ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
coef      0.720      -0.269       0.066       0.379      -0.157       0.083   
p         0.284       0.709       0.922       0.350       0.708       0.830   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
coef      1.225       0.490       0.855       0.564       0.335       0.458  
p         0.011       0.305       0.070       0.061       0.261       0.122


=== Gibbs results (posterior kappa mean) ===


target        pi_cpi                         pi_cpi_core              \
Epi_spec         Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp   
param                                                                  
markup_BN_inv  0.034       0.083       0.077       0.039       0.073   

target                    pi_pce                         pi_pce_core  \
Epi_spec      Epi_spf_cpi    Epi Epi_spf_gdp Epi_spf_cpi         Epi   
param                                                                  
markup_BN_inv       0.072 -0.043      -0.006      -0.022      -0.007   

target                                 
Epi_spec      Epi_spf_gdp Epi_spf_cpi  
param                                  
markup_BN_inv       0.007       0.001


=== Gibbs results (posterior theta mean) ===


target   pi_cpi                         pi_cpi_core                          \
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi   
param                                                                         
N_BN      0.414      -0.187       0.012       0.281      -0.118       0.061   

target   pi_pce                         pi_pce_core                          
Epi_spec    Epi Epi_spf_gdp Epi_spf_cpi         Epi Epi_spf_gdp Epi_spf_cpi  
param                                                                        
N_BN      0.854       0.329       0.591       0.439       0.262       0.358